# Code to measure the offset of each MIRI galaxy to the centre of the frame

To do this, we will read in the catalogue of galaxies to obtain their respective IDs, RAs and Decs. Then we will fit a centroid to all our MIRI images and determine their coordinates. From this we can calculate the spherical offsets. Lastly we will plot the results and see what happens.

The code is separated into several functions that are being called by one main function for better readability and easier debugging.

In [ ]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
from astropy.nddata import Cutout2D
from photutils import centroids
import pandas as pd
import scipy
import os
import glob

Define the necessary functions:

In [ ]:

def load_cutout(file_path, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_path) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None, None

def compute_centroid(cutout, smooth_sigma, good_frac_cutout, smooth_miri):
    """Computes the centroid using a smoothed version of the image."""
    if cutout is None:
        return None

    # Decide whether to smooth MIRI or not
    if smooth_miri == True:
        smoothed_data = scipy.ndimage.gaussian_filter(cutout.data, smooth_sigma)
    else: 
        smoothed_data = cutout.data
    
    # Makes sure the boxsize is an odd number
    search_boxsize = int(np.floor(good_frac_cutout * cutout.shape[0]) // 2 * 2 + 1)

    centroid_pix = centroids.centroid_quadratic(
        smoothed_data,
        xpeak=cutout.shape[0] // 2,
        ypeak=cutout.shape[1] // 2,
        search_boxsize=search_boxsize,
        fit_boxsize=5
    )

    return cutout.wcs.pixel_to_world(centroid_pix[0], centroid_pix[1]) if not np.isnan(centroid_pix).any() else None

def save_alignment_figure(g, cutout_nircam, cutout_miri, centroid_nircam, centroid_miri, output_dir, survey, filter):
    """Saves a figure showing centroid alignment."""
    fig, axs = plt.subplots(1, 2, figsize=[10, 5])

    axs[0].imshow(scipy.ndimage.gaussian_filter(cutout_nircam.data, 1.0), origin='lower')
    axs[0].plot(*cutout_nircam.wcs.world_to_pixel(centroid_nircam), 'x', color='red')
    axs[0].set(title=f"NIRCam F444W Reference {g['id']}")

    axs[1].imshow(scipy.ndimage.gaussian_filter(cutout_miri.data, 1.0), origin='lower')
    axs[1].plot(*cutout_miri.wcs.world_to_pixel(centroid_miri), 'o', color='orange')
    axs[1].set(title=f"MIRI {filter} Cutout {g['id']}")
    
    # show expected position of the centroid
    expected_position_pix = cutout_miri.wcs.world_to_pixel(centroid_nircam)
    axs[1].plot(expected_position_pix[0], expected_position_pix[1], 'x', color='red')

    output_path = os.path.join(output_dir, f"{g['id']}_{filter}_offset_{survey}.pdf")
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(output_path)
    plt.close()

    #print(f"Saved figure: {output_path}")


def compute_offset(cutout_folder, survey, filter, obs="", smooth_miri=True):
    """Computes the astrometric offset between NIRCam and MIRI for each galaxy."""

    global global_cat
    
    for i, g in enumerate(global_cat):
        #print(f"Processing galaxy {g['id']}...")

        ref_position = SkyCoord(ra=g['ra'], dec=g['dec'], unit=u.deg)
        cutout_size = (2.5 * u.arcsec, 2.5 * u.arcsec)
        smooth_sigma, good_frac_cutout = 1.0, 0.7

        if g['id'] == 21451: # exclude bright source nearby
            good_frac_cutout = 0.4
        if g['id'] == 9986: # exclude bright source nearby
            good_frac_cutout = 0.4
        if g['id'] == 11451: # exclude bright source nearby
            good_frac_cutout = 0.4
        
        # Load MIRI cutout
        cutout_miri_path = os.path.join(cutout_folder, f"{g['id']}_{filter}_cutout_{survey}{obs}_rot.fits")
        miri_data, miri_wcs = load_cutout(cutout_miri_path)
        if miri_data is None:
            continue
        cutout_miri = Cutout2D(miri_data, ref_position, cutout_size, wcs=miri_wcs)

        # Load NIRCam cutout
        nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{g['id']}_F444W_cutout.fits"
        nircam_data, nircam_wcs = load_cutout(nircam_path)
        if nircam_data is None:
            continue
        cutout_nircam = Cutout2D(nircam_data, ref_position, cutout_size, wcs=nircam_wcs)

        # Compute centroids
        centroid_nircam = compute_centroid(cutout_nircam, smooth_sigma, good_frac_cutout, smooth_miri)
        centroid_miri = compute_centroid(cutout_miri, smooth_sigma, good_frac_cutout, smooth_miri)

        if centroid_nircam is None or centroid_miri is None:
            print("Centroid not found for one or both cutouts. Skipping.")
            continue

        # Save alignment figure
        output_dir = f"/home/bpc/University/master/Red_Cardinal/offsets/{survey}{obs}/"
        save_alignment_figure(g, cutout_nircam, cutout_miri, centroid_nircam, centroid_miri, output_dir, survey, filter)

        # Compute offsets
        dra, ddec = centroid_nircam.spherical_offsets_to(centroid_miri)
        global_cat[f'{survey}{obs}_dra'][i] = dra.to(u.arcsec).value
        global_cat[f'{survey}{obs}_ddec'][i] = ddec.to(u.arcsec).value

        #print(f"Offset: ΔRA = {dra.to(u.arcsec)}, ΔDec = {ddec.to(u.arcsec)}")



Define the global catalogue:

In [ ]:
catalogue = '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

global_cat = Table.read(catalogue)

# Add the columns to store the astrometric offsets
for col in ["primer003_dra", "primer003_ddec", 
            "primer004_dra", "primer004_ddec", 
            "cweb1_dra", "cweb1_ddec",
            "cweb2_dra", "cweb2_ddec"]:
    if col not in global_cat.colnames:
        global_cat[col] = 0.0 * u.arcsec

global_cat

Load the necessary data files

In [ ]:

# Specify the path to the cutouts directory
my_cutouts = '/home/bpc/University/master/Red_Cardinal/cutouts/'
co_rotated = '/home/bpc/University/master/Red_Cardinal/cutouts_rotated/'


In [ ]:
compute_offset(co_rotated, 'primer', 'F770W', '003')
compute_offset(co_rotated, 'primer', 'F770W', '004')
compute_offset(co_rotated, survey='cweb', filter='F770W', obs="1")
compute_offset(co_rotated, survey='cweb', filter='F770W', obs="2")

Now we visualise the catalogue and store it in a csv-file

In [ ]:
global_cat
fname = "/home/bpc/University/master/Red_Cardinal/offsets/astrometric_offsets.csv"
global_cat.write(fname, format="csv", overwrite=True)

# Function to visualise the offsets

In [ ]:
def visualise_offsets(df, col1, output_dir, exclude_ids, outlier_thresh):
    """
    Produces three types of plots for the astrometric offsets (Scatter, Quiver, and Histogram)
    and returns the filtered DataFrame for further analysis.

    Args:
        df (pandas DataFrame): The complete dataframe with all offsets stored.
        col1 (str): The name of the first column of the sub-survey ('dra').
        output_dir (str): Path to the output directory.
        exclude_ids (list[int]): A list of galaxy IDs to be excluded from analysis.
        outlier_thresh (float): A threshold for the maximum degree of scatter before an offset is considered an outlier.

    Returns:
        pandas DataFrame: The filtered DataFrame for further analysis.
    """

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Exclude specific galaxy IDs
    df = df[~df['id'].isin(exclude_ids)].copy()

    # Find corresponding ddec column
    col2 = col1.replace("dra", "ddec")

    # Remove rows where col1 is exactly 0.0
    df_new = df[df[col1] != 0.0].copy()

    # Determine survey name
    survey = 'PRIMER' if 'primer' in col1 else 'COSMOS-Web'

    # ---- Compute Statistics ----
    mean_dra, std_dra = df_new[col1].mean(), df_new[col1].std()
    mean_ddec, std_ddec = df_new[col2].mean(), df_new[col2].std()

    # Print and save statistics
    stats_text = (
        f"{survey} Astrometric Offset Statistics\n"
        f"-----------------------------------\n"
        f"Mean ΔRA  : {mean_dra:.4f} arcsec\n"
        f"Std  ΔRA  : {std_dra:.4f} arcsec\n"
        f"Mean ΔDec : {mean_ddec:.4f} arcsec\n"
        f"Std  ΔDec : {std_ddec:.4f} arcsec\n\n"
    )
    
    print(stats_text)
    
    stats_path = "/home/bpc/University/master/Red_Cardinal/offsets/offset_statistics.txt"
    with open(stats_path, "a") as f:
        f.write(stats_text)

    # ---- Scatter Plot ----
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.scatter(df_new[col1], df_new[col2], s=10, alpha=0.7)
    ax.set_xlabel('ΔRA (arcsec)')
    ax.set_ylabel('ΔDec (arcsec)')
    ax.set_title(f'{survey} Astrometric Offset\nF770W MIRI vs F444W NIRCam')

    scatter_path = os.path.join(output_dir, col1.replace('dra', 'offset_scatter.png'))
    fig.savefig(scatter_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

    # ---- Quiver Plot ----
    fig, ax = plt.subplots(figsize=(6, 5))
    
    ax.quiver(df_new['ra'], df_new['dec'], df_new[col1], df_new[col2], angles='xy', scale_units='xy', scale=1)
    ax.set_xlabel('RA')
    ax.set_ylabel('Dec')
    ax.set_title(f'{survey} Astrometric Offset\nF770W MIRI vs F444W NIRCam')

    # Calculate and adjust axis limits to fit all arrows
    ra_min, ra_max = df_new['ra'].min(), df_new['ra'].max()
    dec_min, dec_max = df_new['dec'].min(), df_new['dec'].max()
    arrow_max = np.sqrt(df_new[col1]**2 + df_new[col2]**2).max()

    ax.set_xlim(ra_min - arrow_max, ra_max + arrow_max)
    ax.set_ylim(dec_min - arrow_max, dec_max + arrow_max)

    quiver_path = os.path.join(output_dir, col1.replace('dra', 'offset_arrows.png'))
    fig.savefig(quiver_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

    # ---- Histogram Plot ----
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    
    axs[0].hist(df_new[col1], bins=15, edgecolor='black', alpha=0.7)
    axs[0].set_title("ΔRA (arcsec)")
    axs[0].set_xlabel("Offset (arcsec)")

    axs[1].hist(df_new[col2], bins=15, edgecolor='black', alpha=0.7)
    axs[1].set_title("ΔDec (arcsec)")
    axs[1].set_xlabel("Offset (arcsec)")

    hist_path = os.path.join(output_dir, col1.replace('dra', 'offset_histogram.png'))
    fig.savefig(hist_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    # Rename columns for consistency
    df_new = df_new.rename(columns={col1: 'dra', col2: 'ddec'})
    
    return df_new


Call the function!

In [ ]:
# Read in the data as a pandas dataframe
fname = "/home/bpc/University/master/Red_Cardinal/astrometric_offsets.csv"
df = pd.read_csv(fname)

# Exclude the following galaxies from analysis
exclude_ids = [19098, 19681, 21451, 7934, 8465, 9517, 10415, 11247, 11451, 12133,
               12175, 12213, 7696, 9809, 10600, 11137, 16615, 16874, 17517, 11481,
               12443, 20720, 21472, 21547, 22606]

# Specify output directory
output_dir = "/home/bpc/University/master/Red_Cardinal/offsets/plots/"

# Specify path of the statistics.txt file
stat_file = "/home/bpc/University/master/Red_Cardinal/offsets/offset_statistics.txt"

# Check if file exists, then remove it
if os.path.exists(stat_file):
    os.remove(stat_file)
    print(f"Removed existing file: {stat_file}")

# Call the function and store new DataFrames
df_primer003 = visualise_offsets(df, 'primer003_dra', output_dir, exclude_ids, 0.0)
df_primer004 = visualise_offsets(df, 'primer004_dra', output_dir, exclude_ids, 0.0)
df_cweb1 = visualise_offsets(df, 'cweb1_dra', output_dir, exclude_ids, 0.0)
df_cweb2 = visualise_offsets(df, 'cweb2_dra', output_dir, exclude_ids, 0.0)


Produce scatter plots with all surveys included

In [ ]:
figname = output_dir + 'offset_scatter.png'

plt.scatter(df_primer003['dra'], df_primer003['ddec'], label='PRIMER 003')
plt.scatter(df_primer004['dra'], df_primer004['ddec'], label='PRIMER 004')
plt.scatter(df_cweb1['dra'], df_cweb1['ddec'], label='COSMOS-Web 1')
plt.scatter(df_cweb2['dra'], df_cweb2['ddec'], label='COSMOS-Web 2')
plt.xlabel('Delta RA (arcsec)')
plt.ylabel('Delta dec (arcsec)')
plt.title('Astrometric offset from F444W cutout to F770W')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.legend()
plt.axvline(0, ls='--', color='k')
plt.axhline(0, ls='--', color='k')
plt.savefig(figname)
plt.show()


figname = output_dir + 'avg_offset_scatter.png'

plt.scatter(np.mean(df_primer003['dra']), np.mean(df_primer003['ddec']), label='PRIMER 003')
plt.scatter(np.mean(df_primer004['dra']), np.mean(df_primer004['ddec']), label='PRIMER 004')
plt.scatter(np.mean(df_cweb1['dra']), np.mean(df_cweb1['ddec']), label='COSMOS-Web 1')
plt.scatter(np.mean(df_cweb2['dra']), np.mean(df_cweb2['ddec']), label='COSMOS-Web 2')
plt.xlabel('Delta RA (arcsec)')
plt.ylabel('Delta dec (arcsec)')
plt.title('Average astrometric offset from F444W cutout to F770W')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.legend()
plt.axvline(0, ls='--', color='k')
plt.axhline(0, ls='--', color='k')
plt.savefig(figname)
plt.show()


# Define a function that compensates for the offsets

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
import os

def shift_miri_fits(fits_file, dra_mean, ddec_mean, output_dir):
    """
    Shifts the WCS of a MIRI fits file to compensate for a systematic astrometric offset.

    Args:
        fits_file (str): Path to the input MIRI fits file.
        dra_mean (float): Mean RA offset in arcseconds.
        ddec_mean (float): Mean Dec offset in arcseconds.
        output_dir (str): Directory in which the corrected fits file is saved.

    Returns:
        None
    """
    
    # Open the FITS file
    with fits.open(fits_file, mode='update') as hdul:
        hdr = hdul[0].header  # Get the header
        wcs = WCS(hdr)  # Get the WCS from the header

        # Convert RA and Dec shifts from arcsec to degrees
        dra_shift = dra_mean / 3600.0
        ddec_shift = ddec_mean / 3600.0

        # Adjust CRVAL (reference coordinate in degrees)
        hdr['CRVAL1'] -= dra_shift  # RA shift
        hdr['CRVAL2'] -= ddec_shift  # Dec shift

        # Save the updated fits file
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, os.path.basename(fits_file).replace('_rot.fits', '_corr.fits'))
        hdul.writeto(output_file, overwrite=True)

    print(f"Shifted MIRI FITS file saved to: {fits_file}")



Shift the MIRI .fits files:

In [ ]:

# Define input path
miri_rotated = "/home/bpc/University/master/Red_Cardinal/MIRI_rotated/"

# Define output path
corr_dir = "/home/bpc/University/master/Red_Cardinal/MIRI_corrected/"

# Define average offsets based on previous analysis -> maybe automatise
primer003_dra = -0.2207
primer003_ddec = 0.3978

primer004_dra = 0.4170
primer004_ddec = -0.0523

cweb1_dra = 0.3681
cweb1_ddec = 0.0459

cweb2_dra = -0.2084
cweb2_ddec = 0.3868





# Shift primer observation 003
primer003_rot = glob.glob(os.path.join(miri_rotated, "PRIMER_003_rot/*.fits"))
print(f"Found {len(primer003_rot)} rotated FITS files for the PRIMER survey.")
for fits_file in primer003_rot:
    output_dir = corr_dir + "PRIMER_003_corr/"
    shift_miri_fits(fits_file, primer003_dra, primer003_ddec, output_dir)

# Shift PRIMER observation 004
primer004_rot = glob.glob(os.path.join(miri_rotated, "PRIMER_004_rot/*.fits"))
print(f"Found {len(primer004_rot)} rotated FITS files for the PRIMER survey.")
for fits_file in primer004_rot:
    output_dir = corr_dir + "PRIMER_004_corr/"
    shift_miri_fits(fits_file, primer004_dra, primer004_ddec, output_dir)


# Shift the first COSMOS-Web tiles
cweb1_rot = glob.glob(os.path.join(miri_rotated, "COSMOS-Web_1_rot/*.fits"))
print(f"Found {len(cweb1_rot)} rotated FITS files for the COSMOS-Web survey #1.")
for fits_file in cweb1_rot:
    output_dir = corr_dir + "COSMOS-Web_1_corr/"
    shift_miri_fits(fits_file, cweb1_dra, cweb1_ddec, output_dir)

# Shift the second COSMOS-Web tiles
cweb2_rot = glob.glob(os.path.join(miri_rotated, "COSMOS-Web_2_rot/*.fits"))
print(f"Found {len(cweb2_rot)} rotated FITS files for the COSMOS-Web survey #2.")
for fits_file in cweb2_rot:
    output_dir = corr_dir + "COSMOS-Web_2_corr/"
    shift_miri_fits(fits_file, cweb2_dra, cweb2_ddec, output_dir)

